## **7. Creación de un Agente de Inteligencia Artificial para las Conclusiones Finales**

In [ ]:
print("\n7. Creación de un Agente de Inteligencia Artificial para las Conclusiones Finales")

### **7.1. Lectura del Prompt y Configuración del Logging**

In [ ]:
print("\n\t7.1. Lectura del Prompt y Configuración del Logging")
print("\n\t\tProceso iniciado")

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def cargar_prompt():
    prompt_path = 'agente IA/input/prompt/prompt.txt'
    try:
        
        with open(prompt_path, "r", encoding="utf-8") as f:
            text = f.read()
            plain_text = striprtf.rtf_to_text(text)

    except FileNotFoundError:
        logger.error(f"No se encontró el archivo de prompt en {prompt_path}")
        
        default_prompt = f"""
        INSTRUCCIONES:
        Analiza los datos de alojamientos en {ciudad} para {numero_total_personas} personas entre {fecha_entrada_str} y {fecha_salida_str}.
        
        DATOS DISPONIBLES:
        1. Archivo CSV con información detallada de alojamientos
        2. Imágenes con análisis económico: diagrama de cajas e histograma
        3. Imagen con análisis de servicios disponibles
        
        FORMATO DE RESPUESTA REQUERIDO:
        1. Resumen Ejecutivo (máximo 200 palabras)
        2. Análisis Económico
           - Rango de precios
           - Relación calidad-precio
           - Tendencias identificadas
        3. Análisis de Servicios
           - Servicios más comunes
           - Servicios diferenciadores
        4. Recomendaciones
           - Top 3 opciones recomendadas con justificación
        5. Conclusiones
        
        NIVEL DE DETALLE: Exhaustivo con datos cuantitativos específicos
        """

        return default_prompt
    
    except Exception as e:
        
        logger.error(f"Error al cargar el prompt: {e}")
        return f"Generar un análisis completo de los alojamientos en {ciudad}."

prompt = cargar_prompt()

In [ ]:
print("\n\t\tProceso finalizado")

### **7.2. Copia de Imágenes**

In [ ]:
print("\n\t7.2. Copia de Imágenes")
print("\n\t\tProceso iniciado")

In [ ]:
imagenes = list()

origen_imagen_economia_1 = f'output/Análisis Económico/Diagramas de Cajas/Diagrama Caja - {ciudad}.png'
origen_imagen_economia_2 = f'output/Análisis Económico/Histogramas/Histograma - {ciudad}.png'
origen_imagen_servicios = f'output/Análisis de Servicios/Servicios - {ciudad}.png'

imagenes.append(origen_imagen_economia_1)
imagenes.append(origen_imagen_economia_2)
imagenes.append(origen_imagen_servicios)

destino_imagenes = "agente IA/input/images/"

for imagen in imagenes:
    if os.path.exists(imagen):
        shutil.copy(imagen, destino_imagenes)
    else:
        print(f"Imagen no encontrada: {imagen}")

origen_tabla_datos = f"output/Análisis de Datos/Alojamientos. {ciudad}. {numero_total_personas} Personas. {fecha_entrada_str} | {fecha_salida_str}.csv"

destino_data = "agente IA/input/data/"

if os.path.exists(origen_tabla_datos):
    shutil.copy(origen_tabla_datos, destino_data)
else:
    print(f"Tabla de datos no encontrada: {origen_tabla_datos}")

In [ ]:
print("\n\t\tProceso finalizado")

### **7.3. Cargar Varibales de Entorno**

In [ ]:
print("\n\t7.3. Cargar Varibales de Entorno")
print("\n\t\tProceso iniciado")

In [ ]:
def cargar_variables_entorno():
    # Obtener directorio actual y construir ruta absoluta
    current_dir = os.path.dirname(os.path.abspath("agente IA/.env"))
    dotenv_path = os.path.join(current_dir, '.env')
    
    # Verificar existencia del archivo .env
    if not os.path.exists(dotenv_path):
        logger.error(f"Archivo .env no encontrado en {dotenv_path}")
        return False
    
    # Cargar con ruta explícita
    load_dotenv(dotenv_path=dotenv_path)
    
    # Verificar variables críticas
    OLLAMA_MODEL = os.getenv("OLLAMA_MODEL")
    OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")
    
    if not OLLAMA_MODEL:
        logger.error("Variable OLLAMA_MODEL no definida en .env")
        return False
    
    if not OLLAMA_BASE_URL:
        logger.error("Variable OLLAMA_BASE_URL no definida en .env")
        return False
    
    logger.info(f"Variables de entorno cargadas. Modelo: {OLLAMA_MODEL}")
    return True

if not cargar_variables_entorno():
    logger.error("Error al cargar variables de entorno. Utilizando valores predeterminados.")
    os.environ["OLLAMA_MODEL"] = "llama2"
    os.environ["OLLAMA_BASE_URL"] = "http://localhost:11434"

OLLAMA_MODEL = os.getenv("OLLAMA_MODEL")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")

In [ ]:
print("\n\t\tProceso finalizado")

### **7.4. Lectura de Imágenes y Procesamiento de Imágenes y Datos**

In [ ]:
print("\n\t7.4. Lectura de Imágenes")
print("\n\t\tProceso iniciado")

In [ ]:
def verificar_directorios(dirs):
    for dir in dirs:
        if not os.path.exists(dir):
            logger.warning(f"Directorio no encontrado: {dir}")
            os.makedirs(dir, exist_ok=True)
            logger.info(f"Directorio creado: {dir}")
        
        # Verificar si hay archivos en el directorio
        if len(os.listdir(dir)) == 0:
            logger.warning(f"Directorio vacío: {dir}")

directorio_images = "agente IA/input/images/"
directorio_data = "agente IA/input/data/"

verificar_directorios([directorio_images, directorio_data])

try:
    images = SimpleDirectoryReader(directorio_images).load_data()
    logger.info(f"Imágenes cargadas: {len(images)} archivos")
except Exception as e:
    logger.error(f"Error al cargar imágenes: {e}")
    images = []

try:
    data = SimpleDirectoryReader(directorio_data).load_data()
    logger.info(f"Datos cargados: {len(data)} archivos")
except Exception as e:
    logger.error(f"Error al cargar datos: {e}")
    data = []


In [ ]:
directorio_images = "agente IA/input/images/"
directorio_data = "agente IA/input/data/"

images = SimpleDirectoryReader(directorio_images).load_data()
data = SimpleDirectoryReader(directorio_data).load_data()

In [ ]:
print("\n\t\tProceso finalizado")

### **7.5. Inicialización del Almacenamiento en Chroma**

In [ ]:
print("\n\t7.5. Inicialización del Almacenamiento en Chroma")
print("\n\t\tProceso iniciado")

In [ ]:
def inicializar_chroma():
    try:
        chroma_path = "./agente IA/chroma_store"
        os.makedirs(chroma_path, exist_ok=True)
        
        chroma_client = chromadb.PersistentClient(path=chroma_path)
        collection = chroma_client.get_or_create_collection("rag_collection")
        vector_store = ChromaVectorStore(chroma_collection=collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        return storage_context
    except Exception as e:
        logger.error(f"Error al inicializar Chroma: {e}")
        logger.info("Fallback a SimpleVectorStore")
        return StorageContext.from_defaults(vector_store=SimpleVectorStore())

storage_context = inicializar_chroma()

In [ ]:
print("\n\t\tProceso finalizado")

### **7.6. Configuración de LLM y Embedings**

In [ ]:
print("\n\t7.6. Configuración de LLM y Embedings")
print("\n\t\tProceso iniciado")

In [ ]:
def configurar_llm():
    try:
        # Configurar LLM con Ollama y timeout aumentado
        llm = Ollama(
            model=OLLAMA_MODEL,
            base_url=OLLAMA_BASE_URL,
            request_timeout=360.0,  # Timeout aumentado
            temperature=0.1  # Temperatura baja para respuestas más deterministas
        )
        
        # Configurar embedding con timeout adecuado
        embed_model = OllamaEmbedding(
            model_name=OLLAMA_MODEL,
            base_url=OLLAMA_BASE_URL,
            request_timeout=120.0
        )
        
        # Verificar conexión con el servidor de Ollama
        try:
            # Intenta una operación simple para verificar conexión
            llm.complete("test")
        except Exception as e:
            logger.error(f"Error al conectar con el servidor de Ollama: {e}")
            logger.error("Verifica que el servidor de Ollama esté en ejecución y accesible")
            raise
        
        # Configurar Settings
        Settings.llm = llm
        Settings.embed_model = embed_model
        
        return True
    except Exception as e:
        logger.error(f"Error al configurar LLM y embeddings: {e}")
        return False

if not configurar_llm():
    logger.error("No se pudo configurar el LLM. Verifique la conexión con Ollama.")

In [ ]:
print("\n\t\tProceso finalizado")

### **7.7. Creación de un Índice Vectorial**

In [ ]:
print("\n\t7.7. Creación de un Índice Vectorial")
print("\n\t\tProceso iniciado")

In [ ]:
def crear_indice_vectorial(documents):
    if not documents:
        logger.error("No hay documentos para indexar")
        return None
    
    try:
        # Configurar el índice con parámetros optimizados
        storage_context = StorageContext.from_defaults(vector_store=SimpleVectorStore())
        
        index = VectorStoreIndex.from_documents(
            documents=documents,
            storage_context=storage_context
        )
        
        logger.info("Índice vectorial creado exitosamente")
        return index
    except Exception as e:
        logger.error(f"Error al crear índice vectorial: {e}")
        return None

documents = images + data
if not documents:
    logger.warning("No se han cargado documentos (imágenes o datos). El índice estará vacío.")

index = crear_indice_vectorial(documents)
if not index:
    logger.error("No se pudo crear el índice vectorial. No se puede continuar.")
    # Puedes decidir si detener la ejecución o crear un índice vacío como fallback
    index = VectorStoreIndex.from_documents([], StorageContext.from_defaults(vector_store=SimpleVectorStore()))

In [ ]:
print("\n\t\tProceso finalizado")

### **7.8. Creación de una Respuesta**

In [ ]:
print("\n\t7.8. Creación de una Respuesta")
print("\n\t\tProceso iniciado")

In [ ]:
def configurar_parametros_respuesta(complejidad='media'):
    if complejidad == 'baja':
        Settings.context_window = 8192
        Settings.num_output = 2048
        Settings.chunk_size_limit = 4096
    elif complejidad == 'alta':
        Settings.context_window = 32768
        Settings.num_output = 8192
        Settings.chunk_size_limit = 16384
    else:  # media
        Settings.context_window = 20480
        Settings.num_output = 4086
        Settings.chunk_size_limit = 8192
    
    Settings.chunk_overlap_ratio = 0.15  # Aumentado ligeramente para mejor coherencia

def generar_respuesta(index, prompt, max_intentos=3):
    configurar_parametros_respuesta(complejidad='media')
    
    # Mejorar el prompt con instrucciones explícitas
    prompt_mejorado = f"""
    {prompt}
    
    INSTRUCCIONES ADICIONALES:
    - Analiza a fondo las imágenes y datos proporcionados.
    - Identifica patrones, tendencias y anomalías en los datos.
    - Proporciona análisis cuantitativos específicos con cifras y porcentajes.
    - Estructura tu respuesta claramente según el formato solicitado.
    - Incluye recomendaciones justificadas basadas en los datos.
    """
    
    for intento in range(1, max_intentos + 1):
        try:
            logger.info(f"Intento {intento} de generación de respuesta")
            query_engine = index.as_query_engine(
                response_mode="compact",
                similarity_top_k=5  # Usar más documentos para mayor contexto
            )
            response = query_engine.query(prompt_mejorado)
            
            # Validar la respuesta
            if len(str(response)) < 200:
                logger.warning(f"La respuesta es demasiado corta ({len(str(response))} caracteres). Reintentando...")
                continue
                
            logger.info(f"Respuesta generada exitosamente ({len(str(response))} caracteres)")
            return response
        except Exception as e:
            logger.error(f"Error en intento {intento}: {e}")
            if intento == max_intentos:
                logger.error("Se alcanzó el número máximo de intentos sin éxito")
                return "No se pudo generar una respuesta. Error: " + str(e)
            # Esperar un poco antes de reintentar
            import time
            time.sleep(2)

response = generar_respuesta(index, prompt)

In [ ]:
print("\n\t\tProceso finalizado")

### **7.9. Guardar Respuesta**

In [ ]:
print("\n\t7.9. Guardar Respuesta")
print("\n\t\tProceso iniciado")

In [ ]:
def save_output(text, path="output.txt"):
    # Crear el directorio si no existe
    directory = os.path.dirname(path)
    if directory:  # Verifica que el directorio no sea una cadena vacía
        os.makedirs(directory, exist_ok=True)
    
    try:
        # Guardar el archivo
        with open(path, "w", encoding="utf-8") as f:
            f.write(str(text))
        logger.info(f"Respuesta guardada exitosamente en: {path}")
        return True
    except Exception as e:
        logger.error(f"Error al guardar la respuesta: {e}")
        return False

output_path = f"output/Análisis IA/Respuesta IA - {ciudad}.txt"
if save_output(response, output_path):
    pass
else:
    logger.error(f"Error al guardar la respuesta: {e}")

In [ ]:
print("\n\t\tProceso finalizado")